In [3]:
from flask import Flask, request
import socket

app = Flask(__name__)

# Define the Authoritative Server (AS) address and port
AS_IP = '10.9.10.2'
AS_PORT = 53533

@app.route('/fibonacci')
def userServer():
    # Check if all required parameters are present in the request
    required_params = ['hostname', 'fs_port', 'number', 'as_ip', 'as_port']
    for param in required_params:
        if param not in request.args:
            return 'Missing parameter: ' + param, 400
    
    # Get the parameters from the request
    hostname = request.args['hostname']
    fs_port = request.args['fs_port']
    number = request.args['number']
    as_ip = request.args['as_ip']
    as_port = request.args['as_port']
    
    # Query the Authoritative Server (AS) to get the IP address of the given hostname
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    query = 'TYPE=A\nNAME=' + hostname + '\n'
    s.sendto(query.encode(), (AS_IP, AS_PORT))
    response, _ = s.recvfrom(2048)
    response_str = response.decode()
    response_lines = response_str.strip().split('\n')
    response_params = {}
    for line in response_lines:
        key, value = line.split('=')
        response_params[key] = value
    
    # Check if the response from the AS is valid
    required_response_params = ['NAME', 'VALUE', 'TYPE']
    for param in required_response_params:
        if param not in response_params:
            return 'Invalid response from AS', 500
    
    # Get the IP address from the response
    fs_ip = response_params['VALUE']
    
    # Query the Fibonacci Server (FS) to get the Fibonacci number for the given sequence number
    fs_url = 'http://' + fs_ip + ':' + fs_port + '/fibonacci?number=' + number
    fs_response = requests.get(fs_url)
    
    # Return the Fibonacci number if the request was successful
    if fs_response.status_code == 200:
        return fs_response.text, 200
    else:
        return 'Failed to get Fibonacci number', 500

if __name__ == '__main__':
    app.run(port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/yuhanzhang/opt/anaconda3/lib/python3.8/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 48] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhanzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/vy/cfxbck9n24vf_56l8vyxt_sw0000gn/T/ipykernel_21363/1000082152.py", line 57, in <module>
    app.run(port=8080)
  File "/Users/yuhanzhang/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1188, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/Users/yuhanzhang/opt/anaconda3/lib/python3.8/site-packages/werkzeug/serving.py", line 1062, in run_simple
    s = prepare_socket(hostname, port)
  File "/Users/yuhanzhang/opt/anaconda3/lib/pyt

TypeError: object of type 'NoneType' has no len()